In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Question 5

In [ ]:
import csv
import numpy as np
from scipy import stats
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


driver_train_path = "/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/train.csv"
driver_test_path = "/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/test.csv"

# driver_train_path = "train.csv"
# driver_test_path = "test.csv"

In [ ]:
driver_train = pd.read_csv(driver_train_path)
print(driver_train.shape, "- original dataset")
print(driver_train.nunique(dropna = False))

(51490, 42) - original dataset
Report Number                      3256
Local Case Number                  1430
Agency Name                          10
ACRS Report Type                      3
Crash Date/Time                   39573
Route Type                           11
Road Name                          2227
Cross-Street Type                    11
Cross-Street Name                  4526
Off-Road Description               4018
Municipality                         21
Related Non-Motorist                 12
Collision Type                       19
Weather                              13
Surface Condition                    12
Light                                 9
Traffic Control                      12
Driver Substance Abuse               12
Non-Motorist Substance Abuse         10
Person ID                         49209
Injury Severity                       5
Circumstance                        140
Drivers License State                66
Vehicle ID                        49160
Vehicle D

### One hot encodings

In [ ]:
print(driver_train['Non-Motorist Substance Abuse'].value_counts())

NMSA_one_hot = pd.get_dummies(driver_train['Non-Motorist Substance Abuse'])

NMSA_one_hot.drop('ALCOHOL PRESENT, NONE DETECTED', axis='columns', inplace=True)
NMSA_one_hot.drop('N/A, NONE DETECTED', axis='columns', inplace=True)
NMSA_one_hot.drop('NONE DETECTED, UNKNOWN', axis='columns', inplace=True)
NMSA_one_hot.drop('NONE DETECTED', axis='columns', inplace=True)
NMSA_one_hot.drop('UNKNOWN', axis='columns', inplace=True)

driver_train = pd.concat([NMSA_one_hot, driver_train], axis=1)

NONE DETECTED                     1333
N/A, NONE DETECTED                 275
ALCOHOL PRESENT                    275
MEDICATION PRESENT                 267
ILLEGAL DRUG PRESENT               262
UNKNOWN                            258
ALCOHOL PRESENT, NONE DETECTED     248
NONE DETECTED, UNKNOWN             241
ALCOHOL CONTRIBUTED                225
Name: Non-Motorist Substance Abuse, dtype: int64


In [ ]:
print(driver_train['Vehicle First Impact Location'].value_counts())

vfil_one_hot = pd.get_dummies(driver_train['Vehicle First Impact Location'], prefix='VFIL')

vfil_one_hot.drop('VFIL_UNKNOWN', axis='columns', inplace=True)
driver_train = pd.concat([vfil_one_hot, driver_train], axis=1)
driver_train.drop('Vehicle First Impact Location', axis='columns', inplace=True)

TWELVE OCLOCK    19695
SIX OCLOCK       10379
ONE OCLOCK        4713
ELEVEN OCLOCK     3975
TWO OCLOCK        1825
TEN OCLOCK        1815
SEVEN OCLOCK      1391
FIVE OCLOCK       1302
FOUR OCLOCK       1251
EIGHT OCLOCK      1143
NINE OCLOCK       1064
THREE OCLOCK      1042
UNKNOWN            892
UNDERSIDE          310
NON-COLLISION      242
ROOF TOP           229
Name: Vehicle First Impact Location, dtype: int64


In [ ]:
print(driver_train['Vehicle Second Impact Location'].value_counts())

vsil_one_hot = pd.get_dummies(driver_train['Vehicle Second Impact Location'], prefix='VSIL')

vsil_one_hot.drop('VSIL_UNKNOWN', axis='columns', inplace=True)
driver_train = pd.concat([vsil_one_hot, driver_train], axis=1)
driver_train.drop('Vehicle Second Impact Location', axis='columns', inplace=True)

TWELVE OCLOCK    19583
SIX OCLOCK       10252
ONE OCLOCK        4629
ELEVEN OCLOCK     3875
TWO OCLOCK        1886
TEN OCLOCK        1848
SEVEN OCLOCK      1399
FIVE OCLOCK       1310
FOUR OCLOCK       1284
EIGHT OCLOCK      1176
THREE OCLOCK      1135
NINE OCLOCK       1087
UNKNOWN            898
UNDERSIDE          349
ROOF TOP           295
NON-COLLISION      250
Name: Vehicle Second Impact Location, dtype: int64


In [ ]:
print(driver_train['Light'].value_counts())

light_one_hot = pd.get_dummies(driver_train['Light'], prefix='Light')

light_one_hot.drop('Light_DARK -- UNKNOWN LIGHTING', axis='columns', inplace=True)
light_one_hot.drop('Light_OTHER', axis='columns', inplace=True)
light_one_hot.drop('Light_UNKNOWN', axis='columns', inplace=True)
driver_train.drop('Light', axis='columns', inplace=True)


driver_train = pd.concat([light_one_hot, driver_train], axis=1)

DAYLIGHT                    33617
DARK LIGHTS ON              11224
DARK NO LIGHTS               1716
DUSK                         1425
DAWN                         1222
DARK -- UNKNOWN LIGHTING      727
UNKNOWN                       480
OTHER                         375
Name: Light, dtype: int64


In [ ]:
for (columnName, columnData) in driver_train.iteritems():
    if driver_train[columnName].isnull().sum() >= 40000:
      driver_train.drop(columnName, axis='columns', inplace=True)
      
print(driver_train.isnull().sum())

print(driver_train.shape, "- after removing columns with many nulls")


Light_DARK LIGHTS ON        0
Light_DARK NO LIGHTS        0
Light_DAWN                  0
Light_DAYLIGHT              0
Light_DUSK                  0
                        ...  
Equipment Problems      10666
Latitude                    0
Longitude                   0
Location                    0
Fault                       0
Length: 74, dtype: int64
(51490, 74) - after removing columns with many nulls


In [ ]:

# eyeballing the features and removing seemingly unnecessary ones
driver_train.drop('Road Name', axis='columns', inplace=True)
driver_train.drop('Cross-Street Name', axis='columns', inplace=True)
driver_train.drop('Report Number', axis='columns', inplace=True)
driver_train.drop('Person ID', axis='columns', inplace=True)
driver_train.drop('Vehicle ID', axis='columns', inplace=True)
driver_train.drop('Crash Date/Time', axis='columns', inplace=True)
driver_train.drop('Local Case Number', axis='columns', inplace=True)
driver_train.drop('Location', axis='columns', inplace=True)

# filling in missing values after manual inspection
driver_train["Collision Type"].fillna("UNKNOWN", inplace = True)
driver_train["Weather"].fillna("UNKNOWN", inplace = True)
driver_train["Surface Condition"].fillna("UNKNOWN", inplace = True)
driver_train["Traffic Control"].fillna("UNKNOWN", inplace = True)
driver_train["Driver Substance Abuse"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Damage Extent"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Movement"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Movement"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Continuing Dir"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Going Dir"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Make"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Model"].fillna("UNKNOWN", inplace = True)
driver_train["Route Type"].fillna("UNKNOWN", inplace = True)
driver_train["Cross-Street Type"].fillna("UNKNOWN", inplace = True)
driver_train["Latitude"].fillna("UNKNOWN", inplace = True)
driver_train["Longitude"].fillna("UNKNOWN", inplace = True)


# tentative stuff
driver_train.drop('Circumstance', axis='columns', inplace=True)
driver_train["Equipment Problems"].fillna("UNKNOWN", inplace = True)

driver_train["Drivers License State"].fillna("UNKNOWN", inplace = True)
driver_train["Vehicle Body Type"].fillna("UNKNOWN", inplace = True)

driver_train.drop('Vehicle Make', axis='columns', inplace=True)
driver_train.drop('Vehicle Year', axis='columns', inplace=True)
driver_train.drop('Vehicle Model', axis='columns', inplace=True)
driver_train.drop('Agency Name', axis='columns', inplace=True)
driver_train.drop('Latitude', axis='columns', inplace=True)
driver_train.drop('Longitude', axis='columns', inplace=True)


# convert all strings to uppercase and label encoding
le = LabelEncoder()
for (columnName, columnData) in driver_train.iteritems():
  if driver_train.dtypes[columnName] == 'object':
    driver_train[columnName] = driver_train[columnName].str.upper()
    le.fit(driver_train[columnName])
    driver_train[columnName] = le.transform(driver_train[columnName])

print(driver_train.nunique(dropna = False))



print(driver_train.shape, "- manual removal")

driver_train.to_csv('/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/driver_train_m.csv', index=False)

Light_DARK LIGHTS ON       2
Light_DARK NO LIGHTS       2
Light_DAWN                 2
Light_DAYLIGHT             2
Light_DUSK                 2
VSIL_EIGHT OCLOCK          2
VSIL_ELEVEN OCLOCK         2
VSIL_FIVE OCLOCK           2
VSIL_FOUR OCLOCK           2
VSIL_NINE OCLOCK           2
VSIL_NON-COLLISION         2
VSIL_ONE OCLOCK            2
VSIL_ROOF TOP              2
VSIL_SEVEN OCLOCK          2
VSIL_SIX OCLOCK            2
VSIL_TEN OCLOCK            2
VSIL_THREE OCLOCK          2
VSIL_TWELVE OCLOCK         2
VSIL_TWO OCLOCK            2
VSIL_UNDERSIDE             2
VFIL_EIGHT OCLOCK          2
VFIL_ELEVEN OCLOCK         2
VFIL_FIVE OCLOCK           2
VFIL_FOUR OCLOCK           2
VFIL_NINE OCLOCK           2
VFIL_NON-COLLISION         2
VFIL_ONE OCLOCK            2
VFIL_ROOF TOP              2
VFIL_SEVEN OCLOCK          2
VFIL_SIX OCLOCK            2
VFIL_TEN OCLOCK            2
VFIL_THREE OCLOCK          2
VFIL_TWELVE OCLOCK         2
VFIL_TWO OCLOCK            2
VFIL_UNDERSIDE

In [ ]:
driver_test = pd.read_csv(driver_test_path, dtype='unicode')
id = driver_test.loc[:, 'Id']
print(driver_test.shape, " - original dataset")

# one hot encoding
NMSA_one_hot_test = pd.get_dummies(driver_test['Non-Motorist Substance Abuse'])
driver_test = pd.concat([NMSA_one_hot_test, driver_test], axis=1)

vfil_one_hot_test = pd.get_dummies(driver_test['Vehicle First Impact Location'], prefix='VFIL')
driver_test = pd.concat([vfil_one_hot_test, driver_test], axis=1)

vsil_one_hot_test = pd.get_dummies(driver_test['Vehicle Second Impact Location'], prefix='VSIL')
driver_test = pd.concat([vsil_one_hot_test, driver_test], axis=1)

light_one_hot_test = pd.get_dummies(driver_test['Light'], prefix='Light')
driver_test = pd.concat([light_one_hot_test, driver_test], axis=1)


# collecting all the column names from the train set to preserve only those columns in the test set
final_features = driver_train.columns.values
final_features = final_features[:-1]
driver_test = driver_test[final_features]
print(driver_test.shape, " - after dropping irrelevant features")



# filling in missing values after manual inspection
driver_test["Collision Type"].fillna("UNKNOWN", inplace = True)
driver_test["Weather"].fillna("UNKNOWN", inplace = True)
driver_test["Surface Condition"].fillna("UNKNOWN", inplace = True)
driver_test["Traffic Control"].fillna("UNKNOWN", inplace = True)
driver_test["Driver Substance Abuse"].fillna("UNKNOWN", inplace = True)
driver_test["Vehicle Damage Extent"].fillna("UNKNOWN", inplace = True)
driver_test["Vehicle Movement"].fillna("UNKNOWN", inplace = True)
driver_test["Vehicle Movement"].fillna("UNKNOWN", inplace = True)
driver_test["Vehicle Continuing Dir"].fillna("UNKNOWN", inplace = True)
driver_test["Vehicle Going Dir"].fillna("UNKNOWN", inplace = True)
driver_test["Route Type"].fillna("UNKNOWN", inplace = True)
driver_test["Cross-Street Type"].fillna("UNKNOWN", inplace = True)


# tentative stuff
driver_test["Equipment Problems"].fillna("UNKNOWN", inplace = True)

driver_test["Drivers License State"].fillna("UNKNOWN", inplace = True)
driver_test["Vehicle Body Type"].fillna("UNKNOWN", inplace = True)



# convert all strings to uppercase and label encoding
le = LabelEncoder()
for (columnName, columnData) in driver_test.iteritems():
  if driver_test.dtypes[columnName] == 'object':
    driver_test[columnName] = driver_test[columnName].str.upper()
    le.fit(driver_test[columnName])
    driver_test[columnName] = le.transform(driver_test[columnName])

print(driver_test.shape, " - final shape after preprocessing")

driver_test.to_csv('/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/driver_test_m.csv', index=False)

(77235, 42)  - original dataset
(77235, 58)  - after dropping irrelevant features
(77235, 58)  - final shape after preprocessing


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline



driver_train_data = driver_train.to_numpy()
driver_test_data = driver_test.to_numpy()

# Split training/test sets
X = driver_train_data[:, :-1] # all rows, all columns except the last
y = driver_train_data[:, -1]  # all rows, only the last column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

y_train_matrix = y_train.reshape(y_train.shape[0], 1)
y_test_matrix = y_test.reshape(y_test.shape[0], 1)

training_set = np.hstack((X_train, y_train_matrix))
test_set = np.hstack((X_test, y_test_matrix))

training_set = np.asarray(training_set, dtype=np.float32)
test_set = np.asarray(test_set, dtype=np.float32)


In [ ]:
import sklearn.datasets
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from random import randrange
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
import graphviz
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from scipy.stats import uniform, randint

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

clf = GradientBoostingClassifier()
params = {
    "learning_rate": uniform(0.02, 0.2), # default 0.1 
    "max_depth": randint(2, 8), # default 3
    "n_estimators": randint(50, 200), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(clf, param_distributions=params, random_state=0, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X_train, y_train)

report_best_scores(search.cv_results_, 1)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 76.9min finished


Model with rank: 1
Mean validation score: 0.849 (std: 0.001)
Parameters: {'learning_rate': 0.07212573337852397, 'max_depth': 7, 'n_estimators': 168, 'subsample': 0.9834963692226372}



In [ ]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.7, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)

xg_cla.fit(X_train,y_train)
preds = xg_cla.predict(X_test)

accuracy_xgb = accuracy_score(y_test, preds)
print("Accuracy:", accuracy_xgb)
print()

Accuracy: 0.8526574739431605



In [ ]:
# gaussian naive bayes
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

accuracy_gnb = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy_gnb)
print()

Accuracy: 0.705768110312682



Test set

In [ ]:
import xgboost as xgb

xg_cla = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.7, learning_rate = 0.1, max_depth = 8, alpha = 10, n_estimators = 200, subsample=0.7)
xg_cla.fit(X,y)
predictions = xg_cla.predict(driver_test_data)

final_submission = np.c_[id, predictions]
final_submission = np.r_[[['Id', 'Fault']], final_submission]
print(final_submission.shape)
print(final_submission)

(77236, 2)
[['Id' 'Fault']
 ['0' '1']
 ['1' '0']
 ...
 ['77232' '0']
 ['77233' '1']
 ['77234' '1']]


In [ ]:
predictions = predictions.astype(int)
id = id.astype(int)

final_submission = np.c_[id, predictions]
final_submission = np.r_[[['Id', 'Fault']], final_submission]
print(final_submission.shape)
print(final_submission)


# fname = '/content/drive/MyDrive/Acad_Stuff/Course Work/Semester 5/FoML/Hackathon/final_submission.csv'
fname = 'final_submission.csv'

# np.savetxt(fname, final_submission, delimiter=' ')

with open(fname, "w") as f:
    writer = csv.writer(f)
    writer.writerows(final_submission)

(77236, 2)
[['Id' 'Fault']
 ['0' '1']
 ['1' '0']
 ...
 ['77232' '0']
 ['77233' '1']
 ['77234' '1']]


In [ ]:
id = id.astype(int)
temp = np.zeros(id.shape)

temp = temp.astype(int)
id = id.astype(int)


zeros = np.c_[id, temp]
zeros = np.r_[[['Id', 'Fault']], zeros]
print(zeros.shape)
print(zeros)


fname = 'zeros.csv'
# np.savetxt(fname, zeros, delimiter=' ')

with open(fname, "w") as f:
    writer = csv.writer(f)
    writer.writerows(zeros)

(77236, 2)
[['Id' 'Fault']
 ['0' '0']
 ['1' '0']
 ...
 ['77232' '0']
 ['77233' '0']
 ['77234' '0']]
